In [ ]:
import csv
import requests
import math
import asyncio
import time
import itertools
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# API key 세팅
keys = ['90DUYXNC7RT52O6A', 'R520AVI5YQ4GY3KS', 'KAA7D9WHBDPAMI0W', '4OO4HBAVMWBY9SGB', 
    '14YXVYTAAPHMZ08W', 'NQIGNQ11QW6A7U3D', '9XC2SEN7V9PNBI6Q', 'C2B0F4WX5DAI9YU7', 
    'I1PUHU2RCZG4VHTP', '8X22TH1JE6Q3K319', 'JT68VQMI744IBF6N', 'RV6WTJUIHJ2S0IBO', 
    'RKR3ZUG4T6CXOVQW', 'ABL6T0Y6UTZ8OSLK', 'PZFC6U0H39TUFXR3', 'I2BZ8RWBUKIJAE87', 
    'U58CCJM3YPFK7O33', 'ASPPECJOXPREEBJB', 'XQS9KILRSAXGIZZN', 'JH0WHTK54KAUDFGZ', 
    'V0PVJ7MNMWABGGJ3', 'USAG12J9HUXQPKJW', '4HWYPRQZC2ERR5UM', '5EHXMUAABDRHYTFU', 
    '2HXRSCIBK0W1QCC3', '88R2BYU2UY1SA300', '33I85SJ8PUWNN9V9', 'OMA6UXTC238349UC', 
    '3O5XU38XF17VABIT', 'HCD6BXPJ97IZ9VSX', '0HUOIOH7EH4J1YGB', 'PG6TU7KMR6KJ89FB']
api_keys = itertools.cycle(keys[2:])
print("# of API keys:", len(keys))

In [ ]:
# 타겟 종목 조회
def find_valid_symbols(api_keys):
    api_key = next(api_keys)
    api_url = "https://www.alphavantage.co/query?" + "function=LISTING_STATUS" + "&apikey=" + 'api_key'
    with requests.Session() as s:
        download = s.get(api_url)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    data_list = list(cr)
    data_np = np.array(data_list)
    symbols = data_np[1:,0]
    print("# of symbols:", len(symbols))
    
    # 1년 이상의 데이터를 가지고 있는 symbol만 추출
    api_function = 'function=TIME_SERIES_INTRADAY_EXTENDED'
    api_interval = '&interval=60min'
    api_slice = '&slice=year2month1'

    valid_symbols = []
    i_symbol = 0
    print("Index\tSymbol\tValid_data")
    while i_symbol < len(symbols):
        api_key = '&apikey=' + next(api_keys)
        symbol = symbols[i_symbol]
        api_symbol = "&symbol=" + symbol
        api_url = 'https://www.alphavantage.co/query?' + api_function + api_symbol + api_interval + api_slice + api_key
        with requests.Session() as s:
            try:
                download = s.get(api_url)
            except Exception as e:
                print(e)
                time.sleep(10)
                continue
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        data_list = list(cr)
        data_np = np.array(data_list)
        if data_np.shape[1] != 6:
            #print(i_symbol, "api limit")
            time.sleep(60)
            continue

        print(str(i_symbol)+"\t"+symbol+"\t"+str(len(data_np)))
        if len(data_np) > 1:
            valid_symbols.append(symbol)
        i_symbol += 1
    
    return valid_symbols

In [ ]:
# 종목 캔들 가져오기
def get_ohlcv(symbol, data_type, api_keys):
    api_function = 'function=TIME_SERIES_INTRADAY_EXTENDED'
    api_symbol = "&symbol=" + symbol
    api_interval = '&interval=1min'
    #api_slice = '&slice=year1month1'

    ohlcv_data = np.zeros((0,5))
    year_month = [[i, j] for i in range(1,3) for j in range(1,13)]

    if data_type == "train":
        for i_del in range(5):
            del year_month[0]
    elif data_type == "test":
        year_month = [[i, j] for i in range(1,2) for j in range(1,6)]

    i_ym = 0
    while(i_ym < len(year_month)):
        api_key = '&apikey=' + next(api_keys)
        ym = year_month[i_ym]
        i_year = ym[0]
        i_month = ym[1]
        api_slice = '&slice=year' + str(i_year) + 'month' + str(i_month)
        api_url = 'https://www.alphavantage.co/query?' + api_function + api_symbol + api_interval + api_slice + api_key
        with requests.Session() as s:
            try:
                download = s.get(api_url)
            except Exception as e:
                print(e)
                time.sleep(1)
                continue
            decoded_content = download.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            data_list = list(cr)
            data_np = np.array(data_list)

            if data_np.shape[1] != 6:
                #print(i_ym, "api limit")
                time.sleep(1)
                continue

            data_np = data_np[1:,1:6].astype(float)
            ohlcv_data = np.append(ohlcv_data, data_np, axis=0)
            i_ym += 1
        
    ohlcv_data = np.flip(ohlcv_data)
    print(ohlcv_data.shape)

    return ohlcv_data

In [ ]:
# 데이터 다운로드 후 저장
def download_stock_data(api_keys, symbols, mode):          
    save_dir = "../"
    for i, symbol in enumerate(symbols):
        if i < 0:
            continue
        try:
            print(i, symbol)
            if mode == "train":
                ohlcv_data = get_ohlcv(symbol, "train", api_keys)
                np.save(save_dir + "/Data/stock_total/" + symbol + "_minute_.npy", ohlcv_data[:])
            elif mode == "test":
                ohlcv_data = get_ohlcv(symbol, "test", api_keys)
                np.save(save_dir + "/Data/stock_tests/" + symbol + "_minute_.npy", ohlcv_data[:])

        except Exception as e:
            continue

In [ ]:
# 1년 이상 데이터 있는 종목 찾기
symbols = find_valid_symbols(api_keys)

In [ ]:
# 주식 데이터 다운로드
symbols = ['AAPL','MSFT','AMZN','GOOGL','FB','BRK-A','TSLA','TSM','BABA','V','JPM','NVDA','JNJ','WMT','UNH',
           'MA','PYPL','PG','HD','DIS','BAC','ADBE','ASML','CMCSA','NKE','TM','ORCL','KO','XOM','NFLX','VZ',
           'CSCO','PFE','INTC','LLY','CRM','PEP','ABT','ABBV','NVS','TMO','DHR','T','ACN','MRK','AVGO','CVX',
           'TMUS','UPS','WFC']
download_stock_data(api_keys, symbols, "train")
download_stock_data(api_keys, symbols, "test")